# Nanostructures in Equilibrium with Python

© Prof. Branislav K. Nikolić & Dr. Marko D. Petrović, University of Delaware

[PHYS824: Nanophysics & Nanotechnology](https://wiki.physics.udel.edu/phys824) 

## What is covered in this notebook:
- Charge and spin density in 1D nanowire with or without impurities
- DOS of disordered 1D nanowire via eigenvalue binning and Anderson localization of its eigenstates
- DOS of 1D closed or open nanowire via Green functions
- Equilibrium density matrix of open 1D nanowire via Green functions and Ozaki contour complex integration
- Hofstadter butterfly energy spectrum of electrons on a square lattice in magnetic field
- DOS of graphene flake via Green functions

In [ ]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Charge and spin density in 1D nanostructures

### 1.1 Charge density
Compute electron charge density for a 1D nanowire via equilibrium grand canonical density matrix. The nanowire is modeled using [discretized continuous Hamiltonian](https://wiki.physics.udel.edu/phys824/Discretization_of_1D_continuous_Hamiltonian) with possible impurity potential on its diagonal:

$\hat{H} =  \sum_{i=1}^n \big[(U_i+2t)|i\rangle \langle i| - t|i\rangle \langle i+1| -t|i+1 \rangle \langle i| \big]$

Identical tight-binding Hamiltonian (without $2t$ on the diagonal) appears also in modeling of [1D chain of atoms](https://wiki.physics.udel.edu/wiki_phys824/images/8/87/PHYS824_lecture3_atom_to_1D_nanowires.pdf) with one orbital included per atom.

__NOTE: In order for this example to run correctely, all the cells have to be executed one after another.__

Define physical constants

In [ ]:
HBAR = 1.055e-34       # Planck constant [J*s]
M_EL = 9.110e-31       # Electron mass [kg]
Q_EL = 1.602e-19       # Electron charge [C]
mu = 0.25              # Chemical potential 
kbT = 0.025;           # Temperature in eV; useful to remember is that room temperature T=300 K is 0.025eV

and lattice properties

In [ ]:
a = 2e-10                           # Lattice spacing in meters 
n = 100                             # Number of discrete lattice points 
x = np.arange(0, n*a, a)            # Lattice site positions
t = (HBAR**2)/(2*M_EL*(a**2))/Q_EL  # Hopping energy 

Load Hamiltonian matrix representation

$\hat{H} = \begin{bmatrix} 2t  &  -t  &  0  & \cdots & 0 \\
                           -t  &  2t  &  -t  & \cdots & 0 \\
                           0   &  -t  &  2t & \cdots & 0 \\
                           \cdots & \cdots & \cdots & 2t & -t \\
                           0 & \cdots & \cdots & -t & 2t
           \end{bmatrix}$

into NumPy 2D array via shortcut offered by `numpy.diag`

In [ ]:
ham = 2*t*np.diag(np.ones(n))       # Create diagonal elements 
ham += -t*np.diag(np.ones(n-1), -1) # Add lower diagonal hoppings
ham += -t*np.diag(np.ones(n-1), 1)  # Add upper diagonal hoppings

Check shape

In [ ]:
ham.shape

Introduce possible onsite potential and/or periodic boundary conditions

In [ ]:
# Uncomment to add periodic boundary conditions
# ham[0, -1] = -t
# ham[-1, 0] = -t
pot = np.zeros((n, n))
# Uncomment to add impurity in the middle
# pot_impurity = np.zeros(n)
# pot_impurity[int(n/2)] += 0.1
# pot = np.diag(pot_impurity)


Check that Hamiltonian is Hermitian matrix

In [ ]:
la.norm(ham-np.conjugate(ham.T))

Compute eigenvalues and eigenvectors

In [ ]:
evals, evecs = np.linalg.eigh(ham + pot)

Find equilibrium density matrix

$\hat{\rho}_\mathrm{eq} = f(\hat{H} - \mu \hat{I})$

as the Fermi function of the Hamiltonian

In [ ]:
rho = 1. / (1 + np.exp((evals-mu)/kbT))

In [ ]:
from functools import reduce
rho1 = reduce(np.matmul, [evecs, np.diag(rho), np.conj(evecs.T)]) 
density = np.diag(rho1) / a 

Plot the results

In [ ]:
import matplotlib as mpl            # These two lines are for setting DPI 
mpl.rcParams['figure.dpi'] = 100

plt.plot(x, density)
plt.xlabel(r'$x$ (m)')
plt.ylabel(r'Electron density (/$m^3$)')
plt.show()

For running this as standalone Python code, we can put all cells above into a function

In [ ]:
import numpy as np
from functools import reduce

HBAR = 1.055e-34       # Planck constant [J*s]
M_EL = 9.110e-31       # Electron mass [kg]
Q_EL = 1.602e-19       # Electron charge [C]

def adj(ar):
    return np.conj(ar.T)

def density(n=100, mu=0.025, u_impurity=0.0, 
            kbT=0.025, periodic=False):
    a = 2e-10                           # Lattice spacing in meters
    x = np.arange(0, n*a, a)            # Lattice site positions
    # t = (HBAR**2)/(2*M_EL*(a**2))/Q_EL  # Hopping energy 
    t = 1
    # Define Hamiltonian
    ham = 2*t*np.diag(np.ones(n))       # Create diagonal elements 
    ham += -t*np.diag(np.ones(n-1), -1) # Add lower diagonal
    ham += -t*np.diag(np.ones(n-1), 1)  # Add upper diagonal
    
    if periodic:
        ham[0, -1] = -t
        ham[-1, 0] = -t
        
    pot_impurity = np.zeros(n)
    pot_impurity[int(n/2)] += u_impurity
    pot = np.diag(pot_impurity)
    
    evals, evecs = np.linalg.eigh(ham + pot)
    rho = 1. / (1 + np.exp((evals-mu)/kbT))
    
    rho1 = reduce(np.matmul, [evecs, np.diag(rho), np.conj(evecs.T)]) 
    density = np.diag(rho1) / a  
    return x, evals, density

Define plotting function 

In [ ]:
def plot_density(x, density):
    %matplotlib inline
    from matplotlib import pyplot as plt 
    import matplotlib as mpl            # These two lines are for setting DPI 
    mpl.rcParams['figure.dpi'] = 100

    plt.plot(x, density)
    plt.xlabel(r'$x$ (m)')
    plt.ylabel(r'Electron density (/$m^3$)')
    plt.show()

Test the function by comparing results with those obtained by executing individual cells above

In [ ]:
x, evals, dens = density(n=100, mu=0.25, u_impurity=0.0,
                         kbT=0.0025, periodic=False)
plot_density(x, dens)

### 1.2 Spin density 

For the same 1D nanowire, let us compute equilibrium electron spin density in the presence of __classical magnetic impurity__ within the wire. First, our tight-binding Hamiltonian has to be extended into spinfull Hilbert space, $\mathcal{H}_\mathrm{orbital} \otimes \mathcal{H}_\mathrm{spin}$, of a single electron

$\hat{H}= \big[-t\sum_{i=1}^n \big[|i\rangle\langle i+1| + |i+1\rangle\langle i|)\big] \otimes \hat{I}_\mathrm{spin}$

Here the unit operator $\hat{I}_\mathrm{spin}$ in spin space will become just `np.eye(2)` in Python. Second, we add magnetic impurity at one site, as described by its classical magnetization vector $\mathbf{m}=(\sin\theta\cos\phi,\sin \theta \sin\phi,\cos \theta)$, which interacts with quantum spin of an electron via the so-called $sd$ exchange interaction 

$\hat{H} = \left[-t\sum_{i=1}^n (|i\rangle\langle i+1| + |i+1\rangle\langle i|)\right] \otimes \hat{I}_\mathrm{spin} - |p\rangle\langle p| \otimes J_{sd} \hat{\boldsymbol{\sigma}} \cdot \mathbf{m}$

Treating magnetic impurity spin as classical vector means we are assuming its spin is large. Note that the eigenvalue of spin operatore squared in quantum mechanics $\hat{\mathbf{S}}^2$ is $S(S+1)=S^2(1+1/S)$ which deviates from "classical spin" $S^2$ by that term $1/S$ which becomes smaller and smaller correction as $S \rightarrow \infty$. 

In [ ]:
# Define Pauli matrices for spin-1/2 of electron:
SIGMA_X = np.array([[0, 1],
                    [1, 0]])
SIGMA_Y = np.array([[0, -1j],
                    [1j, 0]])
SIGMA_Z = np.array([[1, 0],
                    [0, -1]])
UNITY = np.array([[1, 0],
                  [0, 1]])


def density_matrix(n=100, mu=0.025,  kbT=0.025,  periodic=False, 
                   moment=(1, 0, 0), jsd=0.1,
                   mag_impurity_index=49):
    
    a = 2e-10                  # Lattice spacing in meters
    x = np.arange(0, n*a, a)   # Lattice site positions
 
    # Define Hamiltonian
    ham = 2*t*np.diag(np.ones(n, dtype=complex))       # Create diagonal elements 
    ham += -t*np.diag(np.ones(n-1), -1)                # Add lower diagonal
    ham += -t*np.diag(np.ones(n-1), 1)                 # Add upper diagonal
    if periodic:                           # Add periodic boundary conditions
        ham[0, -1] = -t
        ham[-1, 0] = -t
    ham = np.kron(ham, UNITY) # Make tight-binding Hamiltonian spinfull
    
    # Add magnetic impurity potential with impurity magnetization pointing along (mx,my,mz) vector
    mm = np.array(moment) / np.linalg.norm(moment) # Impurity magnetization vector should be a unit vector
    mx, my, mz = mm
    pot = -jsd * (mx*SIGMA_X + my*SIGMA_Y + mz*SIGMA_Z)
    ind = 2*mag_impurity_index
    ham[ind:ind+2, ind:ind+2] += pot
    
    # Find equilibrium density matrix
    evals, evecs = np.linalg.eigh(ham)
    rho_zero = 1. / (1 + np.exp((evals-mu)/kbT))
    rho = reduce(np.matmul, [evecs, np.diag(rho_zero), np.conj(evecs.T)]) 
    rho /= a
    
    return x, rho

Function to compute spin and charge density

In [ ]:
def get_density(rho):
    n_sites = int(rho.shape[0]/2)
    Tr = np.trace         # If a function name is too long
    mul = np.matmul       # you can rename it
   
    charge = []
    sx = []
    sy = []
    sz = []
    for i in range(n_sites):
        rho_site = rho[2*i:2*i+2, 2*i:2*i+2]
        charge.append(Tr(mul(rho_site, UNITY)))
        sx.append(Tr(mul(rho_site, SIGMA_X)))
        sy.append(Tr(mul(rho_site, SIGMA_Y)))
        sz.append(Tr(mul(rho_site, SIGMA_Z)))
    
    # Convert lists into numpy arrays
    charge = np.array(charge)
    sx = np.array(sx)
    sy = np.array(sy)
    sz = np.array(sz)
    return charge.real, sx.real, sy.real, sz.real

Function to plot multiplot figures with matplotlib

In [ ]:
import matplotlib as mpl            # These two lines are for setting DPI 
mpl.rcParams['figure.dpi'] = 400
#%matplotlib inline

def plot_4panel(x, n, sx, sy, sz):
    fig, axs = plt.subplots(2, 2)
    axs[0, 0].plot(x, n, color='black', lw=0.7)
    axs[0, 0].set_title(r'$n$ vs. $x$')
    axs[0, 0].set_xlabel(r'$x$ (m)')
    axs[0, 0].set_ylabel(r'Electron density ($m^{-1}$)')

    axs[0, 1].plot(x, sx, color='C0', lw=0.7)
    axs[0, 1].set_title(r'${\rm S}_{\rm x}$ vs. $x$')
    axs[0, 1].set_xlabel(r'$x$ (m)')
    axs[0, 1].set_ylabel(r'Spin density ($m^{-1}$)')

    axs[1, 0].plot(x, sy, color='C2', lw=0.7)
    axs[1, 0].set_title(r'${\rm S}_{\rm y}$ vs. $x$')
    axs[1, 0].set_xlabel(r'$x$ (m)')
    axs[1, 0].set_ylabel(r'Spin density ($m^{-1}$)')

    axs[1, 1].plot(x, sz, color='C3', lw=0.7)
    axs[1, 1].set_title(r'${\rm S}_{\rm z}$ vs. $x$')
    axs[1, 1].set_xlabel(r'$x$ (m)')
    axs[1, 1].set_ylabel(r'Spin density ($m^{-1}$)')

    fig.tight_layout(h_pad=1, w_pad=0.2)       # Use this to control the spacing

    plt.show()

Test function for different orientations of impurity magnetization $\mathbf{m}$

In [ ]:
x, rho = density_matrix(n=100, mu=0.25,  kbT=0.025,  periodic=False, 
                        moment=(1, 1, 1), jsd=-2.0, mag_impurity_index=49)
n, sx, sy, sz = get_density(rho)
plot_4panel(x, n, sx, sy, sz)

## 2. DOS of 1D disordered nanowire via eigenvalue binning

We use random on-site potential $U_i \in [-W/2,W/2]$ on every site of tight-binding Hamiltonian:

$\hat{H} =  \sum_{i=1}^n \big[U_i|i\rangle \langle i| -t |i\rangle \langle i+1| -t |i+1 \rangle \langle i| \big]$


where increasing $W$ yields an example of [Anderson localization](https://physicstoday.scitation.org/doi/10.1063/1.3206091) of wavefunctions in 1D lattice which is manifestation of quantum intereference effects. 

In [ ]:
from numpy.random import rand

def disorder_1d(n=500, t=1, periodic=True, w=0.1):
    

    # Hamiltonian matrix for clean wire
    ham = np.diag(np.zeros(n))          # Create diagonal elements 
    ham += -t*np.diag(np.ones(n-1), -1) # Add lower diagonal
    ham += -t*np.diag(np.ones(n-1), 1)  # Add upper diagonal
    
    if periodic:
        ham[0, -1] = -t
        ham[-1, 0] = -t

    # Add Anderson disorder = uniform random variable as
    # on-site potential on every atom
    pot = w*t    # Anderson disorder strength

    # Impurity potential is uniform random variable in the interval [-W/2,W/2]
    u = -0.5*pot + pot*rand(n)
    ham = ham + np.diag(u)
    
    evals, evecs = np.linalg.eigh(ham)

    return evals, evecs


In [ ]:
evals, evects = disorder_1d(w=5.0)

fig, ax = plt.subplots(1, 3)
ax[0].plot(evals)
ax[0].set_title('Energy Eigenvalues')
ax[0].set_xlabel('Eigenvalue No.')
ax[0].set_ylabel('Energy (eV)')

ax[1].hist(evals, bins=80)
ax[1].set_ylabel('DOS')
ax[1].set_xlabel('Energy (eV)')

ax[2].plot(evects[:, 220], color='C3', lw=0.3) # Eigenfunction for state 220
ax[2].set_title('$\Psi(x)$ for Eigenvalue No. 220')
ax[2].set_xlabel('$x$')
ax[2].set_ylabel(r'$\Psi(x)$')

fig.tight_layout(h_pad=1, w_pad=0.2)       # Use this to control the spacing

plt.show()

## 3. DOS of 1D nanowire via Green functions

### 3.1 Closed 1D nanowire

The following function computes DOS in a finite or infinite 1D nanowire via Green functions. The DOS 
$${\rm DOS}(E) = -\frac{1}{\pi} {\rm Tr}\left[{\rm Im}\mathbf{G}^{r}(E)\right]$$
is computed using retarded Green function
$$ \mathbf{G}^{r}(E) = \left[(E + i\eta)\mathbf{I} - \mathbf{H}\right]^{-1}$$
whose imaginary part is
$$ \mathrm{Im}\mathbf{G}^r = (\mathbf{G}^{r}-\mathbf{G}^{a})/2i$$

In [ ]:
from numpy.random import rand
import numpy as np


def dos_closed(n=100, t=1.0, periodic=True, eta=1.e-3,
               emin=-5, emax=5, ne=100):
    """Compute density of states in the nanowire
    
    Parameters
    ----------
    n : int, optional
        Number of sites in the nanowire
    t : float, optional
        Hopping constant between neighboring sites
    periodic : boolean, optional
        If True, set hopping between the first and the last site.
    eta : float, optional
        Small number used to obtain retarded Green's function.
    emin : float, optional
        Lower energy limit for DOS computation
    emax : float, optional
        Upper energy limit for DOS computation
    ne : int, optional
        Number of energy points used in DOS computation
    
    Returns
    -------
    ens : numpy 1D array of floats
        Energies at which the function computes DOS.
    dos : numpy 1D array of floats
        DOS for the given energies.
    """ 

    # Hamiltonian matrix for a clean wire
    ham = np.zeros((n, n), dtype=complex)
    ham += -t*np.diag(np.ones(n-1), -1) # Add lower diagonal
    ham += -t*np.diag(np.ones(n-1), 1)  # Add upper diagonal
    
    if periodic:
        ham[0, -1] = -t
        ham[-1, 0] = -t
        
    energies = np.linspace(emin, emax, int(ne))
    
    dos = []
    for energy in energies:
        green = np.linalg.inv((energy + 1j*eta)*np.eye(n) - ham)
        dos_e = (-1/np.pi)*np.trace((green - np.conj(green.T))/2j)
        dos.append(dos_e)
    
    return energies, np.array(dos)

In [ ]:
import matplotlib.pyplot as plt

en, dos = dos_closed(eta=1.e-1, n=100, periodic=False, ne=500)
plt.plot(en, dos)
plt.xlabel('Energy (t)')
plt.ylabel('DOS')
plt.title('DOS in finite 1D nanowire')
plt.show()

### 3.2 Open 1D nanowire

First we define a function that computes self-energy of a semi-infinite 1D lead from an analytical expression
$$\boldsymbol{\Sigma}^r(E) = 
  \left\{ 
    \begin{array}{lr}
      \frac{t_{\textrm{C}}^2}{2t_{\textrm{L}}^2}
      \left(E - i\sqrt{4t_{\textrm{L}}^2 - E^2}\right), & 
      \textrm{for }|E|\lt 2|t_{\textrm{L}}| \\
      \frac{t_{\textrm{C}}^2}{2t_{\textrm{L}}^2}
      \left(E - \textrm{sgn}(E)
      \sqrt{E^2 - 4t_{\textrm{L}}^2}\right), & 
      \textrm{for }|E|\ge 2|t_{\textrm{L}}| 
    \end{array}
  \right. $$
Here, $t_{\textrm{L}}$ is the hopping between neighboring sites inside of the semi-infinite lead and $t_{\textrm{C}}$ is the hopping between the first site of the lead and the central region as finit length 1D nanowire. The retarded Green function of open wire is then given by
$$\mathbf{G}^{r}(E) = \left[E\mathbf{I} - \mathbf{H} - \boldsymbol{\Sigma}^r_L(E) -\boldsymbol{\Sigma}^r_R(E) \right]^{-1}$$
if both left (L) and right (R) leads are attached. Note that $\boldsymbol{\Sigma}^r_L(E)=\boldsymbol{\Sigma}^r_R(E)=\boldsymbol{\Sigma}^r(E)$ from the expression above, assuming that $t_{\textrm{L}}$ and $t_{\textrm{C}}$ are the same for both L and R leads.

In [ ]:
def sigma(energy, t_couple=1.0, t_lead=1.0):
    """Compute selfenergy of a semi-infinite lead
    
    Parameters
    ----------
    energy : float
       Energy at which to compute the selefenergy.
    t_couple : float, optional
       Hopping between the 1D wire and the
       semi-infinite lead (the default is 1.0)
    t_lead : float, optional
       Hopping between sites withing the 1D 
       semi-infinite lead (the default is 1.0)
    
    Returns
    -------
    se : float
        The selfenergy for the semi-infinite lead
    """
    
    rad = 4*t_lead**2 - energy**2
    coef = t_couple**2 / (2*t_lead**2)
    
    if (rad > 0):
        se = coef * (energy - 1j*np.sqrt(rad)) 
    else:
        se = coef * (energy - np.sign(energy)*np.sqrt(-rad))
    
    return se

Test sigma function by printing its value for energy inside and outside of the band

In [ ]:
print(sigma(1.0))

In [ ]:
from numpy.random import rand
import numpy as np


def dos_open(n=100, t=1.0, t_lead=1.0, t_couple=1.0,
             emin=-5, emax=5, ne=100, attach='both'):
    """Compute density of states in an open 1D nano-wire
    
    Parameters
    ----------
    n : int, optional
        Number of sites in the nanowire.
    t : float, optional
        Hopping constant between neighboring sites.
    t_lead : float, optional
        Hopping in the leads. Both leads have identical hopping.
    t_couple : float, optional
        Hopping in between the leads and the wire.
    attach : string, optional
        Can be set to 
        "left" - attach only left lead, 
        "right" - attach only right lead
        "both" - attach both leads.
    emin : float, optional
        Lower energy limit for DOS computation.
    emax : float, optional
        Upper energy limit for DOS computation.
    ne : int, optional
        Number of energy points used in DOS computation.
    
    Returns
    -------
    ens : numpy 1D array of floats
        Energies at which the function computes DOS.
    dos : numpy 1D array of floats
        DOS for the given energies.
    ldos : numpy 1D array of floats
        LDOS at the central site.
    """ 
    # Hamiltonian matrix for a clean wire
    ham = np.zeros((n, n), dtype=complex)
    n_half = int(n/2)
    ham += -t*np.diag(np.ones(n-1), -1) # Add lower diagonal
    ham += -t*np.diag(np.ones(n-1), 1)  # Add upper diagonal
    
    energies = np.linspace(emin, emax, int(ne))
    
    dos = []
    ldos = []
    
    for energy in energies:
        
        selfenergy = sigma(energy, t_couple, t_lead)
        
        # Pay attention, we are making a copy of ham matrix 
        ham_eff = np.array(ham)       
        
        if attach == 'left':
            ham_eff[0, 0] += selfenergy
        elif attach == 'right':
            ham_eff[-1, -1] += selfenergy
        elif attach == 'both':
            ham_eff[0, 0] += selfenergy
            ham_eff[-1, -1] += selfenergy
        else:
            msg = 'Unknown option %s' % attach
            raise ValueError(msg)
            
        green = np.linalg.inv(energy*np.eye(n) - ham_eff)
        im_g = -(green - np.conj(green.T)) / (2j*np.pi)
        ldos.append(im_g[n_half, n_half])
        dos_e = (-1/np.pi)*np.trace((green - np.conj(green.T))/2j)
        dos.append(dos_e)
    
    return energies, np.array(dos), np.array(ldos)

In [ ]:
def plot_2panel(en, dos, ldos):
    fig, ax = plt.subplots(2, 1)
    ax[0].plot(en, ldos)
    ax[0].set_xlabel('Energy (t)')
    ax[0].set_ylabel('LDOS')
    ax[0].set_title('LDOS in open 1D nanowire')
    
    ax[1].plot(en, dos)
    ax[1].set_xlabel('Energy (t)')
    ax[1].set_ylabel('DOS')
    ax[1].set_title('DOS in open 1D nanowire')

    fig.tight_layout(h_pad=1, w_pad=0.2) 
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

en, dos, ldos = dos_open(n=100, ne=500, attach='right')
plot_2panel(en, dos, ldos)

## 4. Equilibrium density matrix of open 1D nanowire via Green functions and Ozaki contour complex integration

The equilibrium density matrix can be expressed as
\begin{equation}
  \label{egRho}
  \boldsymbol{\rho}_{\rm eq}(E_F) = -{\rm Im}\left[
    \frac{1}{\pi} \int_{-\infty}^{\infty}
    \mathbf{G}^r(E)f(E, E_F)\,dE \right],
\end{equation}
where $G^r(E)$ is the same retarded Green function of an open active region attached to two semi-infinite leads. 
To compute the given integral, one can use the so-called [Ozaki contour](https://doi.org/10.1103/PhysRevB.75.035123). The main idea behind this method is to approximate the Fermi-Dirac function
\begin{equation}
  f(x) = \frac{1}{1 + \exp(x)},
\end{equation}
where the variable $x$ is expressed in terms of the
temperature and the Fermi energy $x = {(E - E_F)} / k_{B}T$.
After aproximation, one has to compute the poles and residues
of the approximate function. The function is approximated by
continued fraction representation
\begin{equation}
  \frac{1}{1+\exp(x)} = \frac{1}{2} - \frac{x}{4}
   \left(\frac{1}{1+\frac{\frac{x^2}{2}}{3+
   \frac{\frac{x^2}{2}}{5 + \ldots}}} \right).
\end{equation}

In the Ozaki method, the equilibrium density matrix is expressed through poles
$\alpha_{p}$ and residues $R_{p}$ of the approximate Fermi-Dirac function is
\begin{equation}
  \boldsymbol{\rho}_{\rm eq}(E_F) = -\frac{1}{\pi}{\rm Im}\left[
                          2\pi i k_{B}T \sum_{p=1}^{N}
                          \mathbf{G}(\alpha_{p})R_{p} \right] + iR\mathbf{G}(iR),
\end{equation}
For continued fraction approximation function, the poles and residues can be computed numericaly. 
The first $N$ poles of function are calculated by
solving the eigenvalue problem of an $N\times N$ matrix $B$ 
\begin{equation}
  B_{n,n+1} = B_{n+1,n} =
  \frac{1}{\sqrt{(2n+1)(2n-1)}},\,\,n\geq1.
\end{equation}
The positive eigenvalues of $B$
\begin{equation}
  B |b_{p}\rangle = b_{p}
  |b_{p}\rangle, \quad b_{p} > 0
\end{equation}
determine the positions of poles in the upper-half
of the complex plane
\begin{equation}
  z_{p} = \frac{1}{b_p}i,
\end{equation}
while the first elements of the corresponding eigenvectors
$|b_{p}\rangle$ determine the residues
\begin{equation}
  R_{p} = \frac{|\langle
  1|b_{p}\rangle|^{2}}{4b_{p}^{2}}.
\end{equation}
When integrating over the energy axis, we must perform a
substitution
\begin{equation}
  z_p = \frac{\alpha_{p} - E_{F}}{k_{B}T},
\end{equation}
from which we obtain
\begin{eqnarray}
  \alpha_{p} & = & E_F + z_{p}k_{B}T \nonumber \\
             & = & E_{F} + \frac{k_{B}T}{b_{p}}i.
\end{eqnarray}

First, we want to define a function that computes Ozaki poles and residues

In [ ]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import eigsh

def ozaki_poles(n_poles):
    '''Compute the first n poles of the Ozaki continued
    fraction representation of the Fermi-Dirac function
    [Phys. Rev. B 75, 035123 (2007)]. The implementation
    presented here is adopted from [Phys. Rev. B 82, 125114 (2010)]

    Parameters
    ----------
    n_poles : int
        Number of poles to compute

    Returns
    -------
    zeros : numpy array of doubles
        An array of n_poles elements with the singular points of the
        Ozaki continued fraction representation
    res : numpy array of doubles 
        An array of residues at positions of zeros.
    '''

    n = n_poles*2 + 1
    x = np.arange(1, n, 1)
    x = 1.0 / (2*np.sqrt(4*x**2 - 1))
    diag_elements = [x, x]

    b = diags(diag_elements, [-1, 1])
    evals, evects = eigsh(b, k=n-1)

    # Function eigsh returns the eigenvectors in columnwise
    # order, so evects[0] is an array of the first elements
    # and not the first eigenvector.
    res = np.abs(evects[0])**2 / (4*evals**2)

    res = res[np.where(evals > 0)]
    evals = evals[np.where(evals > 0)]

    zeros = 1.0 / evals

    # The strange indexing in this part is used just to
    # reverse the ordering in these two arrays
    return zeros[::-1], res[::-1]

Next, we define Green function for 1D chain

In [ ]:
def green_1d(sites=100, energy=0.0, t=1., t_lead=1.,
             t_ls=1., onsite=0.0, u=0.25, imp_index=50):

    h = (-t*(np.eye(sites, k=1) + 
             np.eye(sites, k=-1))).astype(complex)
    
    pot_diag = np.zeros(sites)
    pot_diag[imp_index] = u
    h += np.diag(pot_diag)
    rad = 4*t_lead**2 - energy**2
    self_energy = 0*1j
    if abs(energy.real) < 2*t_lead:
        self_energy = energy - 1j*np.sqrt(rad)
    else:
        if energy.real > 0:
            sgn = +1.0
        else:
            sgn = -1.0
        self_energy = energy - sgn*np.sqrt(-rad)
    self_energy *= (t_ls**2 / (2*t_lead**2))

    h[0, 0] += self_energy
    h[-1, -1] += self_energy

    h += (onsite * np.eye(sites))
    return (np.linalg.inv(energy*np.eye(sites) - h))

In [ ]:
def ozaki(green_func, npoles=20, rd=1.e+20, energy=-3., kbt=0.25):
    
        beta = 1. / (kbt)
        
        poles, res = ozaki_poles(npoles)

        sum_ozaki = 0.0
        for z, r in zip(poles, res):   # zip function is used to iterate over pairs
            pole = energy + 1j*z/beta
            sum_ozaki += (r*green_func(energy=pole))
        sum_ozaki = 2j*sum_ozaki/beta
        rho = (sum_ozaki - np.conjugate(sum_ozaki.T)) / 2j
        moment = 0.5j * rd * green_func(energy=1j*rd)
        
        return rho + moment

In [ ]:
rho_ozaki = ozaki(green_1d, npoles=500, energy=-1.75, kbt=0.025)

In [ ]:
a = 2.e-10
x, evals, dens = density(n=100, mu=0.25, u_impurity=0.25,
                         kbT=0.025, periodic=True)
# Compare Ozaki density with density in periodic closed system
plt.plot(np.diag(rho_ozaki)/a, color='C1', lw=3.5);
plt.plot(dens, color='C0', ls=(1, (2, 2)), lw=3.5);


## 5. Hofstadter butterfly energy spectrum of electrons on square lattice in magnetic field

In [ ]:
import math
from numpy import arange
n=3   # Consider 3x3 square tight-binding lattice with, therefore, 3 atoms in each row 
t = 1 # Nearest-neighbor hopping between atoms
hamhop = np.zeros((n,n)) # Hopping submatrix between different columns of atoms  
hamhop = -t*np.diag(np.ones(n)) # Add hoppings on diagonal

In [ ]:
x_data_plot = []
y_data_plot = []
for phi in arange(-math.pi,math.pi,0.1):

    ham11 = np.zeros((n,n),dtype=complex)     # Submatrix of the first row of atoms
    ham11 += -t*np.exp(-2*1j*phi)*np.diag(np.ones(n-1), -1) # Add lower diagonal of complex hoppings
    ham11 += -t*np.exp(2*1j*phi)*np.diag(np.ones(n-1), 1)  # Add upper diagonal of complex hoppings
    
    ham22 = np.zeros((n,n),dtype=complex)     # Submatrix of the second row of atoms
    ham22 += -t*np.exp(-4*1j*phi)*np.diag(np.ones(n-1), -1) # Add lower diagonal of complex hoppings
    ham22 += -t*np.exp(4*1j*phi)*np.diag(np.ones(n-1), 1)  # Add upper diagonal of complex hoppings
    
    ham33 = np.zeros((n,n),dtype=complex)    # submatrix of the third row of atoms
    ham33 += -t*np.exp(-6*1j*phi)*np.diag(np.ones(n-1), -1) # Add lower diagonal of complex hoppings
    ham33 += -t*np.exp(6*1j*phi)*np.diag(np.ones(n-1), 1)  # Add upper diagonal of complex hoppings
    
    # load submatrices ham11,ham22,ham33,hamhop into the full 9x9 tight-binding Hamiltonian matrix
    ham = np.zeros((9,9),dtype=complex)
    ham[0:3,0:3] += ham11
    ham[3:6,3:6] += ham22
    ham[6:9,6:9] += ham33
    ham[0:3,3:6] += hamhop
    ham[3:6,6:9] += hamhop
    ham[3:6,0:3] += hamhop
    ham[6:9,3:6] += hamhop
    
    eigvals, eigvecs = la.eig(ham)
    
    for j in range(0,9):
        x_data_plot.append(phi)
        y_data_plot.append(eigvals[j].real)

# Add features to our figure
plt.scatter(x_data_plot,y_data_plot,s=10) # s defines the size of points in scatter plot
plt.title('Eigenenergies of 3x3 tight-binding Hamiltonian in magnetic field')
plt.xlabel('Magnetic Flux Ba$^2$ (h/e)')
plt.ylabel('Eigenenergy (eV)')
plt.show()

## 6. DOS of graphene flake via Green functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def dos_graphene(nx=7, ny=4, gamma=1.0, v=0.0,
                 periodic=False, eta=1.e-3, emin=-3.5, emax=3.5, ne=100):
    """Compute density of states in graphene ribbon
    
    Parameters
    ----------
    nx : integer, optional
        Number of carbon atoms along the x-axis.
    ny : integer, optional
        Number of carbon atoms along the y-axis.
    gamma : float, optional
        Hopping between nearest neighbors. The basic unit of energy.
    periodic : boolean, optional
        Introduce periodic boundry conditions along the y-axis.
    v : float, optional
        Onsite potential energy in units of hopping.
    eta : float, optional
        Small number used to obtain retarded Green's function.
    emin : float, optional
        Lower energy limit for DOS computation in the units of hopping.
    emax : float, optional
        Upper energy limit for DOS computation in the units of hopping.
    ne : integer
        Number of points in energy for DOS computation.
    """

    # Define hamiltonians of individual rows of atoms
    # h_11  or    h_22
    # C           C
    # |            
    # C           C
    #             |     
    # C           C       ^ y
    # |                   | 
    # C           C       |
    #                     ------> x
    
    h_size = nx * ny       # Size of the full Hamiltonian matrix
    ham = np.zeros((h_size, h_size), dtype=complex)
    t = -gamma
    v = v*gamma
    
    h_11 = (v*np.diag(np.ones(ny)) +
            t*np.diag(np.ones(ny-1), 1) +
            t*np.diag(np.ones(ny-1), -1))
    h_22 = np.array(h_11)    # Make a copy. Caution! h_22 = h_11 will create an alias
    
    # Remove odd hoppings from h_11
    for i in range(1, ny-1, 2):
        h_11[i, i+1] = 0.
        h_11[i+1, i] = 0.
    
    # Remove even hoppings from h_22
    for i in range(0, ny-1, 2):
        h_22[i, i+1] = 0.
        h_22[i+1, i] = 0.
        
    # Matrices for coupling between two rows of atoms
    h_ij = t*np.diag(np.ones(ny)) 
    
    if periodic:
        # Wrap system along the y-axis
        h_11[0, -1] = t
        h_11[-1, 0] = t
        h_22[0, -1] = t
        h_22[-1, 0] = t
        
        # Wrap system along the x-axis
        ham[:ny, -ny:] = h_ij
        ham[-ny:, :ny] = h_ij
        
    # Put h_11 and h_22 as diagonal blocks of the main hamiltonian
    for i in range(0, nx, 2):
        ham[i*ny:(i+1)*ny, i*ny:(i+1)*ny] = h_11
        
    for i in range(1, nx, 2):
        ham[i*ny:(i+1)*ny, i*ny:(i+1)*ny] = h_22
    
    # Put the coupling matrices as the main off-diagonal blocks
    hx = (np.diag(np.ones(nx-1), 1) 
          + np.diag(np.ones(nx-1), -1))
    ham = ham + np.kron(hx, h_ij)
    
    energies = np.linspace(emin, emax, ne)
    
    dos = []
    for energy in energies:
        green = np.linalg.inv((energy + 1j*eta)*np.eye(h_size) - ham)
        dos_e = (-1/np.pi)*np.trace((green - np.conj(green.T))/2j)
        dos.append(dos_e)
    
    return energies, np.array(dos)

In [ ]:
ens, dos = dos_graphene(nx=20, ny=12, eta=0.1, v=0, periodic=True, ne=500)
plt.plot(ens, dos)
plt.xlabel('Energy (t)')
plt.ylabel('DOS')
plt.title('DOS of graphene flake')
plt.show()

Send corrections to <code>bnikolic@udel.edu</code>